In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./winemag-data-130k-v2.csv", index_col=0)

# Stories

1. Wine Enthusiast Magazine is planning to feature the top 5 countries that produce the highest-rated wines. To determine the countries, they need you to calculate the average rating (points) for each country and identify the top 5 based on the average rating. Additionally, they want to know the total number of wines reviewed from each of these top 5 countries. Provide the results in a well-formatted table.


In [3]:
## MY SOLUTION

ser_top_5_country = df.groupby("country")["points"].mean().sort_values(ascending=False)[:5]
ser_top_5_total_reviews_count = (
    df[
        df["country"].isin(ser_top_5_country.index)
    ]
    .groupby("country")
    .count()
    ["description"] # choose any column
    .rename("total_review_count") # then rename this column
)


pd.merge(
    ser_top_5_country,
    ser_top_5_total_reviews_count,
    left_index=True,
    right_index=True
)

,points,total_review_count
country,,
England,91.581081,74
India,90.222222,9
Austria,90.101345,3345
Germany,89.851732,2165
Canada,89.369650,257


In [4]:
## IDEAL SOLUTION

df.groupby("country").agg(
    average_rating=("points", "mean"),
    total_review_count=("country", "count")
).sort_values(by="average_rating", ascending=False)[:5]


,average_rating,total_review_count
country,,
England,91.581081,74
India,90.222222,9
Austria,90.101345,3345
Germany,89.851732,2165
Canada,89.369650,257


2. A wine collector is interested in investing in wines from the top 3 provinces known for producing the most expensive wines. Identify the top 3 provinces based on the maximum price of wines from each province. For each of these provinces, calculate the average price and the total number of wines reviewed. Present the findings in a readable format.

In [5]:
## MY SOLUTION

(
    df.groupby("province").agg(
        max_price=("price", "max"),
        average_price=("price", lambda x: round(x.mean(), 2)),
        total_number_of_wines_reviewed=("description", "count")
    )
    .sort_values(by="max_price", ascending=False)
    [["average_price", "total_number_of_wines_reviewed"]]
    [:3]
)

,average_price,total_number_of_wines_reviewed
province,,
Bordeaux,38.91,5941
Burgundy,71.06,3980
California,39.04,36247


In [6]:
## IDEAL SOLUTION

(
    df.groupby("province")
      .agg(
          max_price=pd.NamedAgg(column="price", aggfunc="max"),
          average_price=pd.NamedAgg(column="price", aggfunc=lambda x: round(x.mean(), 2)),
          total_wines_reviewed=pd.NamedAgg(column="description", aggfunc="count")
      )
      .nlargest(3, "max_price")
    [["average_price", "total_wines_reviewed"]]
)

,average_price,total_wines_reviewed
province,,
Bordeaux,38.91,5941
Burgundy,71.06,3980
California,39.04,36247


3. A restaurant owner wants to create a wine menu that showcases the best wines from each of the top 5 varieties (grape types). Determine the top 5 varieties based on the count of wines reviewed for each variety. For each of these top 5 varieties, find the wine with the highest rating and display its details, including the winery, country, province, price, and description. Organize the information in a visually appealing way.

In [17]:
## MY SOLUTION

(
    df[
        df["variety"].isin(
            (
                df.groupby("variety")
                    .agg(
                        total_wines_reviewed=("description", "count"),
                        highest_rated_wine_title=("points", "max"),
                    )
                    .nlargest(5, "total_wines_reviewed")
            ).index
        )
    ].nlargest(5, "points")
    [["variety", "winery", "country", "province", "title", "price", "description"]]
    
    .reset_index(drop=True)
)

,variety,winery,country,province,title,price,description
0,Cabernet Sauvignon,Cardinale,US,California,Cardinale 2006 Cabernet Sauvignon (Napa Valley),200.0,Tasted in a flight of great and famous Napa wi...
1,Bordeaux-style Red Blend,Château Léoville Barton,France,Bordeaux,Château Léoville Barton 2010 Saint-Julien,150.0,"This is a magnificently solid wine, initially ..."
2,Chardonnay,Salon,France,Champagne,Salon 2006 Le Mesnil Blanc de Blancs Brut Char...,617.0,This new release from a great vintage for Char...
3,Bordeaux-style Red Blend,Château Lafite Rothschild,France,Bordeaux,Château Lafite Rothschild 2010 Pauillac,1500.0,"Almost black in color, this stunning wine is g..."
4,Bordeaux-style Red Blend,Château Cheval Blanc,France,Bordeaux,Château Cheval Blanc 2010 Saint-Émilion,1500.0,This is the finest Cheval Blanc for many years...


In [18]:
## IDEAL SOLUTION

(
    df.groupby("variety")
      .agg(
          total_wines_reviewed=("title", "count"),
          highest_rated_wine=("points", "idxmax")
      )
      .nlargest(5, "total_wines_reviewed")
      .reset_index()
      .merge(
          df,
          left_on="highest_rated_wine",
          right_index=True
      )
      [["variety_x", "winery", "country", "province", "title", "price", "description"]]
      .rename(columns={"variety_x": "variety"})
      .reset_index(drop=True)
)

,variety,winery,country,province,title,price,description
0,Pinot Noir,Williams Selyem,US,California,Williams Selyem 2009 Precious Mountain Vineyar...,94.0,"A stupendous Pinot Noir, showing how beautiful..."
1,Chardonnay,Salon,France,Champagne,Salon 2006 Le Mesnil Blanc de Blancs Brut Char...,617.0,This new release from a great vintage for Char...
2,Cabernet Sauvignon,Cardinale,US,California,Cardinale 2006 Cabernet Sauvignon (Napa Valley),200.0,Tasted in a flight of great and famous Napa wi...
3,Red Blend,Tenuta San Guido,Italy,Tuscany,Tenuta San Guido 2012 Sassicaia (Bolgheri Sas...,235.0,"One of Italy's most iconic bottlings, the 2012..."
4,Bordeaux-style Red Blend,Château Léoville Barton,France,Bordeaux,Château Léoville Barton 2010 Saint-Julien,150.0,"This is a magnificently solid wine, initially ..."


4.A wine enthusiast group is planning a tasting event featuring the best wines from different price ranges. They define the price ranges as follows: `Budget` (under 10usd), `Mid-Range` (10usd to 50usd), `Premium` (50usd to 100usd), and `Luxury` (above 100usd). For each price range, determine the following:
1. The number of wines in that price range.
2. The average rating of wines in that price range.
3. The top 3 most common varieties in that price range.
4. The country with the highest average rating in that price range. Present the findings in a structured and visually appealing format.

In [181]:
## COMBINED SOLUTION
# Got sstuck on subquestions 3 and 4.

def calculate_price_bucket(price):
    if pd.isnull(price):
        return "N/A"
    if price <= 10:
        return "Budget"
    elif 10 < price <= 50:
        return "Mid-Range"
    elif 50 < price <= 100:
        return "Premium"
    elif 100 < price:
        return "Luxury"

df["price_bucket_name"] = df["price"].apply(lambda price: calculate_price_bucket(price))

# Aggregate the data based on the price buckets
price_bucket_stats = df.groupby('price_bucket_name').agg(
    number_of_wines=('description', 'count'),
    average_rating=('points', lambda points: round(points.mean(), 2)),
    top_3_varieties=('variety', lambda x: ", ".join(
            str(variety) for variety in (
                x.value_counts()
                .nlargest(3)
                .index
            )
        )
    ),
    highest_rated_country=(
        'country',
        lambda x: (
            df.loc[x.index, ["country", "points"]]
            .groupby("country")
            .agg(
                average_points=("points", "mean")
            )
            .nlargest(1, "average_points")
        ).index,
    ),
    highest_rated_points=('points', 'max')
)

price_bucket_stats

,number_of_wines,average_rating,top_3_varieties,highest_rated_country,highest_rated_points
price_bucket_name,,,,,
Budget,6280,85.02,"Chardonnay, Cabernet Sauvignon, Portuguese Red",India,91
Luxury,3366,92.82,"Cabernet Sauvignon, Pinot Noir, Bordeaux-style...",Portugal,100
Mid-Range,94921,88.04,"Chardonnay, Pinot Noir, Red Blend",England,99
N/A,8996,88.79,"Bordeaux-style Red Blend, Chardonnay, Pinot Noir",Hungary,99
Premium,16408,91.05,"Pinot Noir, Cabernet Sauvignon, Chardonnay",Austria,100


5. Wine Enthusiast Magazine wants to feature the top 10 wineries known for producing the highest-rated wines. For each of the top 10 wineries, they require the following information:
   1. The average rating of wines from that winery.
   2. The total number of wines reviewed from that winery.
   3. The most expensive wine from that winery, along with its price and variety.
   4. The country and province where the winery is located.

Provide the information in a well-organized table format.

In [273]:
(
    df.groupby("winery").agg(
        highest_rated=("points", "max"),
        average_ratings=("points", lambda points: round(points.mean(), 2)),
        review_count=("taster_name", "count"),
        most_expensive_wine_idx=("price", "idxmax")
    )
    .sort_values("highest_rated", ascending=False)
    .head(10)
    .reset_index()
    .assign(
        most_expensive_wine_idx= lambda x: x["most_expensive_wine_idx"].astype(int),
        most_expensive_price=lambda x: df.loc[x["most_expensive_wine_idx"], "price"].tolist(),
        most_expensive_variety=lambda x: df.loc[x["most_expensive_wine_idx"], "variety"].tolist(),
        country=lambda x: df.loc[x["most_expensive_wine_idx"], "country"].tolist(),
        province=lambda x: df.loc[x["most_expensive_wine_idx"], "province"].tolist()
    )
    [[
        "winery",
        "average_ratings",
        "review_count",
        "most_expensive_price",
        "most_expensive_variety",
        "country",
        "province"
    ]]
)

,winery,average_ratings,review_count,most_expensive_price,most_expensive_variety,country,province
0,Château Léoville Barton,95.67,9,150.0,Bordeaux-style Red Blend,France,Bordeaux
1,Louis Roederer,93.27,43,600.0,Champagne Blend,France,Champagne
2,Château Lafite Rothschild,93.75,20,1500.0,Bordeaux-style Red Blend,France,Bordeaux
3,Tenuta dell'Ornellaia,96.70,1,460.0,Merlot,Italy,Tuscany
4,Casanova di Neri,95.17,1,270.0,Sangiovese Grosso,Italy,Tuscany
5,Château Cheval Blanc,93.73,11,1500.0,Bordeaux-style Red Blend,France,Bordeaux
6,Cayuse,93.89,81,175.0,Syrah,US,Washington
7,Biondi Santi,92.53,11,900.0,Sangiovese Grosso,Italy,Tuscany
8,Cardinale,96.00,0,300.0,Bordeaux-style Red Blend,US,California
9,Avignonesi,91.29,6,237.0,Prugnolo Gentile,Italy,Tuscany


6. A research team is analyzing the relationship between wine ratings and the length of wine descriptions. They hypothesize that wines with longer descriptions tend to have higher ratings. To test this hypothesis, perform the following tasks:
   1. Calculate the average length (number of words) of wine descriptions for each rating level (80-84, 85-89, 90-94, 95-100).
   2. Determine the correlation coefficient between the rating and the description length.
   3. Identify the top 10 most frequently used words in the descriptions of wines with a rating of 95 or higher.

Present your findings, along with your conclusions on whether the hypothesis holds true based on the analysis.

In [274]:
(
    df.groupby("winery").agg(
        highest_rated=("points", "max"),
        average_ratings=("points", lambda points: round(points.mean(), 2)),
        review_count=("taster_name", "count"),
        most_expensive_wine_idx=("price", lambda price: int(price.idxmax()) if not pd.isnull(price.max()) else 0)
    )
    .sort_values("highest_rated", ascending=False)
    .head(10)
    .reset_index()
    .assign(
        most_expensive_price=lambda x: df.loc[x["most_expensive_wine_idx"], "price"].tolist(),
        most_expensive_variety=lambda x: df.iloc[x["most_expensive_wine_idx"], df.columns.get_loc("variety")].tolist(),
        country=lambda x: df.iloc[x["most_expensive_wine_idx"], df.columns.get_loc("country")].tolist(),
        province=lambda x: df.iloc[x["most_expensive_wine_idx"], df.columns.get_loc("province")].tolist()
    )
    [["most_expensive_wine_idx", "winery", "average_ratings", "review_count", "most_expensive_price", "most_expensive_variety", "country", "province"]]
)

,most_expensive_wine_idx,winery,average_ratings,review_count,most_expensive_price,most_expensive_variety,country,province
0,58352,Château Léoville Barton,95.67,9,150.0,Bordeaux-style Red Blend,France,Bordeaux
1,89739,Louis Roederer,93.27,43,600.0,Champagne Blend,France,Champagne
2,111753,Château Lafite Rothschild,93.75,20,1500.0,Bordeaux-style Red Blend,France,Bordeaux
3,10743,Tenuta dell'Ornellaia,96.70,1,460.0,Merlot,Italy,Tuscany
4,111754,Casanova di Neri,95.17,1,270.0,Sangiovese Grosso,Italy,Tuscany
5,111755,Château Cheval Blanc,93.73,11,1500.0,Bordeaux-style Red Blend,France,Bordeaux
6,16502,Cayuse,93.89,81,175.0,Syrah,US,Washington
7,88870,Biondi Santi,92.53,11,900.0,Sangiovese Grosso,Italy,Tuscany
8,75669,Cardinale,96.00,0,300.0,Bordeaux-style Red Blend,US,California
9,82750,Avignonesi,91.29,6,237.0,Prugnolo Gentile,Italy,Tuscany
